# Data Visualization of Boonsong Lekagul waterways readings.csv using Altair

import all the required libraries for loading and visualising the data. The data from csv file is loaded to a table and then visually analysed. 

In [1]:
import pandas as pd
df = pd.read_csv("Boonsong Lekagul waterways readings.csv")
df['sample date'] = pd.to_datetime(df['sample date']) # date data formating

In [2]:
import altair as alt
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

# Data Exploration
## a. Measure and location wise raw data exploration

In [3]:
# Initially the entire data is visualised in measure wise basis in each loaction. The subplot is drawn by for looping through the measues and then vertically concatinated.

from altair.expr import datum
measure=list(df.measure.unique()) # listing unique mesasure names from the raw data
selection = alt.selection_multi(fields=['location'], bind='legend') # for interactive legend selection
base = alt.Chart(df).mark_line().encode(
x='sample date:T'
, y='value:Q',
tooltip=['location', 'measure', 'sample date', 'value'],
color=alt.Color('location:N', legend=alt.Legend( # customising legend position and direction
        orient='none',
        legendX=-30, legendY=-60,
        direction='horizontal',
        titleAnchor='middle')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
).properties(# customising the subplots height and width
    width=900,
    height=100
).add_selection(
    selection
).interactive()


charts = []
for measure in measure: # looping through the measues
    charts.append(base.transform_filter(datum.measure == measure).properties(
    title=measure))
alt.vconcat(*charts) # vertically concatinated subplots

alt.VConcatChart(...)

## b. Data accuracy checking
### 1.Number of Duplicate data (same location, date, value and messure) per location

In [4]:

# In order to check the accurary of data, the number of duplicates existing in the raw data is analysed visually over the location
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.Chart(df).mark_bar().encode(
    x=alt.X('measure:O', title='Measure'),
    y=alt.Y('number_of_duplicates:Q', title='Number of Duplicates'),
    color=alt.Color('location:N', legend=alt.Legend(
        orient='none',
        legendX=-30, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
).transform_aggregate(#aggregating total duplicate count based on location
    number_of_duplicates='count(df)',
    groupby=['year',"measure","sample date", "value",'location']
).properties(
    title='Number of duplicates per location',
    width=1000,
    height=300
).add_selection(
    selection
).interactive()

alt.Chart(...)

### 2. Number of Duplicate data (same location, date, value and messure) per year

In [5]:
# In order to check the accurary of data, the number of duplicates existing in the raw data is analysed visually over the year

df['year'] =df['sample date'].dt.year
year= list(df.year.unique())
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.Chart(df).mark_bar().encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('number_of_duplicates:Q', title='Number of Duplicates'),
    color=alt.Color('location:N', legend=alt.Legend(
        orient='none',
        legendX=230, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
).transform_aggregate(# aggregating total duplicate count based on year
    number_of_duplicates='count(sample date)',
    groupby=['year',"measure","sample date", "value",'location']
).properties(
    title='Number of duplicates per year',
    width=1000,
    height=300
).add_selection(
    selection
).interactive()

alt.Chart(...)

## c. Data exploration by narrowing the raw data to specific measure and location in yearwise

In [6]:
# In order to understand the data in more depth, the raw data is narrowed to specific year, location 
# current code checking the year-wise duplicate water temperature data for Chai location 
from altair.expr import datum
measure=list(df.measure.unique())
selection = alt.selection_multi(fields=['location'], bind='legend')
year= list(df.year.unique())
charts = []
for year in year: # looping through the year
    above_data = df[df['year']==year]
    base = alt.Chart(above_data).mark_point().encode(
    x='sample date:T'
    , y='value:Q'
    , color=alt.Color('location:N', legend=alt.Legend(
        orient='none',
        legendX=130, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    ).properties(
    width=1000,
    height=150
    ).add_selection(
    selection
    )
    
    charts.append(base.transform_filter(datum.measure=='Water temperature' & datum.location=='Chai').properties(title=str(year)))
alt.vconcat(*charts) # vetical concatination of subplots 

alt.VConcatChart(...)

## d. Data cleaning by removing the duplicates

In [7]:
# Duplicate data is removed using drop_duplicates() method provided by panda dataframe and the accurate data is visualised 
from altair.expr import datum
measure=list(df.measure.unique())
selection = alt.selection_multi(fields=['location'], bind='legend')
year= list(df.year.unique())
charts = []
for year in year:
    a_df = df[df['year']==year]
    df1=a_df[(a_df['measure']=='Water temperature')]
    above_data=df1.drop_duplicates(['sample date'])[['value','location','sample date','measure','year']]# removing duplicate data
    base = alt.Chart(above_data).mark_point().encode(
    x='sample date:T'
    , y='value:Q'
    , color=alt.Color('location:N', legend=alt.Legend(
        orient='none',
        legendX=130, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    ).properties(
    width=900,
    height=150
    ).facet(column='year:O').add_selection(
    selection
    )
    
    charts.append(base.transform_filter(datum.location=='Chai').properties(title=str(year)))
alt.vconcat(*charts)

alt.VConcatChart(...)

# Visual Data Analysis of accurate data

# Question 2. Describe any data quality and uncertain issues, such as
## i. missing data

In [8]:
# Missing data checking for same location but different years
# Copper data of Achara location of 2009 and 2012 is considered for this analysis. This data is collected and available for one data per month
from altair.expr import datum
selection = alt.selection_interval() # selection of type "interval"
year= [2009,2012]
charts = []
for year in year:
    a_df = df[df['year']==year]
    df1=a_df[(a_df['measure']=='Copper')]
    above_data=df1.drop_duplicates(['sample date','value'])[['value','location','sample date','measure','year']] # removving duplicates
    base = alt.Chart(above_data).mark_bar(size=30).encode(
    x='month(sample date):T'
    , y='value:Q'
    , tooltip=['location', 'measure', 'sample date', 'value'] # adding tooltip containing relevent details
    , color=alt.condition(selection, 'location:N', alt.value('lightgray'))
    ).properties(
    width=500,
    height=250
    ).add_selection(
    selection
    ).interactive()
    
    charts.append(base.transform_filter(datum.location=='Achara').properties(title="Copper "+str(year))) # adding transform_filter for filtering the specific location
alt.hconcat(*charts,spacing=100) # horizondal concatination with space between the chart

alt.HConcatChart(...)

In [9]:
# Missing data checking for same year but different locations
from altair.expr import datum
location= ["Somchair","Kannika"]
charts = []
scale = alt.Scale(domain=[2016], range=['firebrick'])
for location in location:
    a_df = df[df['year']==2016]
    df1=a_df[(a_df['measure']=='Sulphates')]
    above_data=df1.drop_duplicates(['sample date','value'])[['value','location','sample date','measure','year']]
    base = alt.Chart(above_data).mark_bar(size=30).encode(
    x='month(sample date):T'
    , y='value:Q'
    , tooltip=['location', 'measure', 'sample date', 'value']
    , color=alt.Color('year:N', scale=scale)
    ).properties(
    width=500,
    height=250
    ) 
    
    charts.append(base.transform_filter(datum.location==location).properties(title="Sulphates - "+str(location))) # adding transform_filter for filtering the specific location
alt.hconcat(*charts,spacing=100)

alt.HConcatChart(...)

## ii. change in collection frequency

In [10]:
# daily and monthly data collected for water temperature in different locations in different years
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
print(real_data.shape)

base = alt.Chart(real_data).mark_point().encode(
x='sample date:T',
y='value:Q',
).properties(
    width=900,
    height=300
    )


upper = base.mark_line(point=alt.OverlayMarkDef(color="#d62728")).transform_filter(
(datum.location == 'Chai') & (datum.year == 2016) & (datum.measure == 'Water temperature') 
).properties(
    title="Water temperature in Chai in 2016")
lower = base.mark_bar(size=40).encode(x=alt.X('month(sample date):T'),color=alt.value("#2ca02c")).transform_filter(
(datum.location == 'Decha') & (datum.year == 2015) & (datum.measure == 'Water temperature') 
).properties(
    title="Water temperature in Decha in 2015")
upper & lower

(117185, 5)


alt.VConcatChart(...)

In [11]:
# monthly twice and monthly once data collected for total nitrogen in different locations Achara in 2012 and Somchair in 2007
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
print(real_data.shape)

base = alt.Chart(real_data).mark_point().encode(
x='sample date:T',
y='value:Q',
).properties(
    width=900,
    height=300
    )

upper = base.mark_bar(size=25).encode(x=alt.X('sample date:T'),color=alt.value("#d6616b")).transform_filter((datum.location == 'Achara') & (datum.year == 2012) & (datum.measure == 'Total nitrogen')).properties(title="Total nitrogen in Achara in 2012")
lower = base.mark_bar(size=25).encode(x=alt.X('month(sample date):T'),color=alt.value("#e377c2")).transform_filter((datum.location == 'Somchair') & (datum.year == 2007) & (datum.measure == 'Total nitrogen')).properties(title="Total nitrogen in Somchair in 2007")
upper & lower

(117185, 5)


alt.VConcatChart(...)

In [12]:
# Multiple collection frequency for a measure Biochemical Oxygen for a location Busarakhan in a selected year 2015
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
alt.Chart(real_data).mark_bar(size=30).encode(
x='month(sample date):T',
y='count(month(sample date)):Q',
color=alt.value("#2ca02c")
).properties(
    width=900,
    height=300,
    title="Multiple collection frequency of Biochemical Oxygen in Busarakhan in 2015"
    ).transform_filter(
(datum.location == 'Busarakhan') & (datum.year == 2015) & (datum.measure == 'Biochemical Oxygen') 
)
 

alt.Chart(...)

## iii. Unrealistic values

In [13]:
# Unrealistic values of different messures over different years and locations

real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
print(real_data.shape)

base = alt.Chart(real_data).mark_point().encode(
x='sample date:T',
y='value:Q',
tooltip=['location', 'measure', 'sample date', 'value']
).properties(
    width=800,
    height=150
    ) 

# customising by using  different color for the points in the connected graph of each subplot
chart1 = base.mark_line(point=alt.OverlayMarkDef(color="#d62728")).transform_filter((datum.location == 'Tansanee') & (datum.year == 2014) & (datum.measure == 'Nitrites')).properties(title="Nitrites in Tansanee in 2014")
chart2 = base.mark_line(point=alt.OverlayMarkDef(color="#843c39")).transform_filter((datum.location == 'Boonsri') & (datum.year == 2006) & (datum.measure == 'Cadmium')).properties(title="Cadmium in Boonsri in 2006")
chart3 = base.mark_line(point=alt.OverlayMarkDef(color="#637939")).transform_filter((datum.location == 'Tansanee') & (datum.year == 2016) & (datum.measure == 'Biochemical Oxygen')).properties(title="Biochemical Oxygen in Tansanee in 2016")
chart4 = base.mark_line(point=alt.OverlayMarkDef(color="#843c39")).transform_filter((datum.location == 'Sakda') & (datum.year == 2014) & (datum.measure == 'Potassium')).properties(title="Potassium in Sakda in 2014")
chart5 = base.mark_line(point=alt.OverlayMarkDef(color="#843c39")).transform_filter((datum.location == 'Sakda') & (datum.year == 2014) & (datum.measure == 'Chemical Oxygen Demand (Mn)')).properties(title="Chemical Oxygen Demand (Mn) in Sakda in 2014")

chart1 & chart2 & chart3 & chart4 & chart5 

(117185, 5)


alt.VConcatChart(...)

# Question 1. Describe trends and anomalies with respect to chemical contamination
## i. Trends: changes over time and/or sensor site

In [14]:
# Pattern of water temperature over different locations over 1998-2016
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']] # droping duplicates
df1=real_data[(real_data['location']=='Boonsri') | (real_data['location']=='Chai') ] [['value','location','sample date','measure','year']]
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.Chart(df1).mark_line().encode(
x='sample date:T'
, y='value:Q',
tooltip=['location', 'measure', 'sample date', 'value'],
color=alt.Color('location:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
).properties(
    width=900,
    height=300,
    title='Pattern of water temperature over different locations over 1998-2016'
).transform_filter(datum.measure == 'Water temperature').add_selection(# filtering water temperature values
    selection
).interactive()

  

alt.Chart(...)

In [15]:
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
df1=real_data[(real_data['location']=='Chai')| (real_data['location']=='Sakda')| (real_data['location']=='Kannika') ] [['value','location','sample date','measure','year']]
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.Chart(df1).mark_bar(size=15).encode(
x='sample date:T'
, y='value:Q',
tooltip=['location', 'measure', 'sample date', 'value'],
color=alt.Color('location:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
).properties(
    width=900,
    height=300,
    title='Trends of Chlorodinine over different location over 2014-2016'
).transform_filter(datum.measure == 'Chlorodinine').add_selection(
    selection
).interactive()

 

alt.Chart(...)

## ii. Anomalies: sudden change over time or one site significantly different from others.

In [16]:
#Anomalies of Total hardness in Boonsri over 2005-2016
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
df1=real_data[(real_data['location']=='Boonsri') ] [['value','location','sample date','measure','year']]
base=alt.Chart(df1).encode(
x='sample date:T'
, y='value:Q',
tooltip=['location', 'measure', 'sample date', 'value'],
color=alt.value("#ad494a")
 ).properties(
    width=900,
    height=350,
    title='Anomalies of Total hardness in Boonsri over 2005-2016'
).transform_filter(datum.measure == 'Total hardness') 

base.mark_line() + base.mark_point() # layered charts

alt.LayerChart(...)

In [17]:
#Anomalies of Atrazines in Busarakhan over 1998-2016
real_data=df.drop_duplicates(['value','location','sample date','measure','year'])[['value','location','sample date','measure','year']]
df1=real_data[(real_data['location']=='Busarakhan') ] [['value','location','sample date','measure','year']]
base=alt.Chart(df1).encode(
x='sample date:T'
, y='value:Q',
tooltip=['location', 'measure', 'sample date', 'value'],
color=alt.value("#637939")
 ).properties(
    width=900,
    height=300,
    title='Anomalies of Atrazines in Busarakhan over 1998-2016'
).transform_filter(datum.measure == 'Atrazine') # filtering Atrazine values

base.mark_line() + base.mark_point() # layered charts

alt.LayerChart(...)